**Connect to Google Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Unzip the data**

In [ ]:
import zipfile
import os

def unzip_file(zip_filepath, extract_to_dir):
    """
    Unzips a zip file to the specified directory.

    :param zip_filepath: Path to the zip file
    :param extract_to_dir: Directory where the files should be extracted
    """
    # Ensure the extraction directory exists
    if not os.path.exists(extract_to_dir):
        os.makedirs(extract_to_dir)

    with zipfile.ZipFile(zip_filepath, 'r') as zip_ref:
        zip_ref.extractall(extract_to_dir)
        print(f"Extracted all files to {extract_to_dir}")

# Example usage
zip_filepath = '/content/drive/MyDrive/Annotation_Prediction.zip'
extract_to_dir = '/content/Bounded_Box_Images'
unzip_file(zip_filepath, extract_to_dir)



Extracted all files to /content/Bounded_Box_Images


**Detect Images that only contain Bounding Boxes, Or being Annotated "Smoke"**,

-> Save these Images in a separate Folder.

In [ ]:
!pip install opencv-python-headless numpy

import cv2
import numpy as np
import os

def is_blue_present(image):
    # Convert the image to HSV color space
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Define the range for the color blue in HSV
    lower_blue = np.array([100, 150, 0])
    upper_blue = np.array([140, 255, 255])

    # Create a mask to detect blue color
    mask = cv2.inRange(hsv, lower_blue, upper_blue)

    # Check if blue is present in the image
    return np.any(mask)

def filter_images_with_blue_bounding_boxes(input_folder, output_folder):
    # Create the output folder if it does not exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Loop through all files in the input folder
    for filename in os.listdir(input_folder):
        # Read the image
        image_path = os.path.join(input_folder, filename)
        image = cv2.imread(image_path)

        if image is None:
            continue

        # Check if the image contains blue bounded boxes
        if is_blue_present(image):
            # Save the image to the output folder
            output_path = os.path.join(output_folder, filename)
            cv2.imwrite(output_path, image)

# Example usage
input_folder = '/content/Bounded_Box_Images'
output_folder = '/content/Bounded_Box_Images_Smoke'
filter_images_with_blue_bounding_boxes(input_folder, output_folder)


**Data Augmentation**

Flipping

Rotation

Translation

Blurring



In [ ]:
import cv2
import numpy as np
import os
import random

def is_blue_present(image):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    lower_blue = np.array([100, 150, 0])
    upper_blue = np.array([140, 255, 255])
    mask = cv2.inRange(hsv, lower_blue, upper_blue)
    return np.any(mask)

def augment_image(image):
    augmented_images = []

    # 1. Original Image
    augmented_images.append(image)

    # 2. Flipping
    augmented_images.append(cv2.flip(image, 1))  # Horizontal flip
    augmented_images.append(cv2.flip(image, 0))  # Vertical flip

    # 3. Rotation
    rows, cols, _ = image.shape
    M = cv2.getRotationMatrix2D((cols/2, rows/2), 90, 1)
    augmented_images.append(cv2.warpAffine(image, M, (cols, rows)))
    M = cv2.getRotationMatrix2D((cols/2, rows/2), 180, 1)
    augmented_images.append(cv2.warpAffine(image, M, (cols, rows)))
    M = cv2.getRotationMatrix2D((cols/2, rows/2), 270, 1)
    augmented_images.append(cv2.warpAffine(image, M, (cols, rows)))

    # 4. Translation
    M = np.float32([[1, 0, 25], [0, 1, 25]])
    augmented_images.append(cv2.warpAffine(image, M, (cols, rows)))
    M = np.float32([[1, 0, -25], [0, 1, -25]])
    augmented_images.append(cv2.warpAffine(image, M, (cols, rows)))


    # 5. Blurring
    blur_image = cv2.GaussianBlur(image, (5, 5), 0)
    augmented_images.append(blur_image)



    return augmented_images

def filter_and_augment_images(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for filename in os.listdir(input_folder):
        image_path = os.path.join(input_folder, filename)
        image = cv2.imread(image_path)

        if image is None or not is_blue_present(image):
            continue

        augmented_images = augment_image(image)

        for i, aug_image in enumerate(augmented_images):
            output_path = os.path.join(output_folder, f"{os.path.splitext(filename)[0]}_aug_{i}.jpg")
            cv2.imwrite(output_path, aug_image)

# Example usage
input_folder = '/content/Bounded_Box_Images_Smoke'
output_folder = '/content/Augmentated_Images'
filter_and_augment_images(input_folder, output_folder)


**Number Of Images After Augmentation**

In [ ]:
import os

def count_images_in_folder(folder_path):
    # List all files in the directory
    files = os.listdir(folder_path)

    # Filter out files that are not images
    image_files = [file for file in files if file.endswith(('.jpg'))]

    # Count the number of image files
    number_of_images = len(image_files)

    return number_of_images

# Example usage
augmented_images_folder = '/content/Augmentated_Images'
number_of_images = count_images_in_folder(augmented_images_folder)
print(f"Number of images in the augmented images folder: {number_of_images}")


Number of images in the augmented images folder: 13527


**Image Segmentation**

Prominent Smoke Area in Images, Using the Concept of ROIs

In [ ]:
import cv2
import numpy as np
import os

def highlight_and_extract_blue_bounded_boxes(image):
    # Convert the image to HSV color space
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Define the lower and upper bounds for the blue color
    lower_blue = np.array([100, 150, 0])
    upper_blue = np.array([140, 255, 255])

    # Create a mask to isolate the blue color
    mask = cv2.inRange(hsv, lower_blue, upper_blue)

    # Find contours in the mask
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Draw bounding rectangles around the blue bounded boxes
    blue_boxes_found = False
    for contour in contours:
        # Calculate the area of the contour
        area = cv2.contourArea(contour)

        # Only consider contours with a certain area (to filter out noise)
        if area > 1000:  # Adjust this threshold as needed
            x, y, w, h = cv2.boundingRect(contour)
            cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
            blue_boxes_found = True

    return blue_boxes_found, image

def extract_and_highlight_images_with_rois(input_folder, output_folder):
    # Create the output folder if it does not exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Iterate over the images in the input folder
    for filename in os.listdir(input_folder):
        # Read the image
        image_path = os.path.join(input_folder, filename)
        image = cv2.imread(image_path)

        # Highlight blue bounded boxes and check if any blue bounded boxes are found
        blue_boxes_found, highlighted_image = highlight_and_extract_blue_bounded_boxes(image)

        # Save the image only if blue bounded boxes are found
        if blue_boxes_found:
            output_image_path = os.path.join(output_folder, filename)
            cv2.imwrite(output_image_path, highlighted_image)

# Specify the input and output folders
input_folder = '/content/Augmentated_Images'
output_folder = '/content/Roissss'

# Perform extraction and highlight blue bounded boxes
extract_and_highlight_images_with_rois(input_folder, output_folder)


In [ ]:

ROIs_images_folder = '/content/Roissss'
number_of_images = count_images_in_folder(ROIs_images_folder)
print(f"Number of images in the ROIs images folder: {number_of_images}")


Number of images in the ROIs images folder: 9574
